# Goal: We want to have the LLM refine the current timebox json why issueing patches to the current json instead of creating a new one.
In the first iteration the LLM generates the timebox json from scratch.
For each iteration, the draft_schedule is validated, if there in a validation error this is returned to the LLM and the LLM is asked to fix the error.
The LLM then issues a patch to the current draft, this patch is applied deterministically and validated, if passes the current draft is shown to the user.
If the user wants to change the draft, the LLM is giving the instructions and the current draft, ot however important to note that we also want to include previous instructions from the user.

In [ ]:
from pydantic import BaseModel, Field, model_validator
from datetime import date, timedelta, datetime
from typing import List
from fateforger.agents.schedular.models.new_models import CalendarEvent

# Make sure CalendarEvent is defined somewhere above or below this cell

class Timebox(BaseModel):
    events: List[CalendarEvent] = Field(default_factory=list)
    date: "date" = Field(default_factory=lambda: date.today() + timedelta(days=1))  # Default to tomorrow

    @model_validator(mode='before')
    def schedule_and_validate(cls, values):
        # Todo: check for non last bg event
        planning_date = values["date"]
        events= values["events"]

        # 1) Forward pass (default flex-forward)
        last_dt: datetime | None = None
        for ev in events:
            # compute two-of-three cases
            if ev.start and ev.duration and ev.end is None:
                ev.end = (datetime.combine(planning_date, ev.start) + ev.duration).time()
            elif ev.end and ev.duration and ev.start is None:
                ev.start = (datetime.combine(planning_date, ev.end) - ev.duration).time()
            elif ev.start and ev.end and ev.duration is None:
                ev.duration = datetime.combine(planning_date, ev.end) - datetime.combine(planning_date, ev.start)

            # flex-forward scheduling
            if ev.start is None and ev.end is None and not ev.flex_back:
                if last_dt is None:
                    raise ValueError(f"{ev.id}: needs start or duration")
                ev.start = last_dt.time()
                ev.end   = (last_dt + ev.duration).time()

            last_dt = datetime.combine(planning_date, ev.end)

        # 2) Backward pass (flex_back events)
        next_dt: datetime | None = None
        for ev in reversed(events):
            if ev.flex_back and ev.start is None and ev.end is None:
                if next_dt is None:
                    raise ValueError(f"{ev.id}: needs end or duration")
                ev.end   = next_dt.time()
                ev.start = (next_dt - ev.duration).time()
            next_dt = datetime.combine(planning_date, ev.start)

        # 3) Overlap check
        for a, b in zip(events, events[1:]):
            dt_a_end = datetime.combine(planning_date, a.end)
            dt_b_start = datetime.combine(planning_date, b.start)
            if dt_a_end > dt_b_start:
                raise ValueError(f"Overlap: {a.id} → {b.id}")

        # 4) Ensure last event’s start is within the planning day
        last_event = events[-1]
        dt_last_start = datetime.combine(planning_date, last_event.start)
        if dt_last_start.date() != planning_date:
            raise ValueError(
                f"{last_event.id}: start {dt_last_start} is not on {planning_date}"
            )

        return values

In [ ]:
from trustcall import create_extractor

# 1. Build the extractor in update mode:
calendar_agent = create_extractor(
    llm,                          # your OpenAI/LLM instance
    tools=[validate_calendar],    # or [CalendarList]
    tool_choice="validate_calendar",  
    enable_updates=True           # turn on the patch-and-retry loop
)

# 2. Invoke it, passing your current JSON state and user instruction:
result = calendar_agent.invoke({
    "messages": "Please move the team sync from 10 AM to 11 AM and shift all downstream events accordingly.",
    "existing": current_calendar_json
})

# 3. Under the hood:
#    a. The LLM is prompted to call the `validate_calendar` tool
#    b. If `existing` fails validation, trustcall asks for a JSON‐Patch diff
#    c. It applies that patch with python‐json‐patch…
